In [1]:
import pandas as pd
import os
import csv
import numpy as np
from scipy import stats
from matplotlib import pyplot as plt
%matplotlib

Using matplotlib backend: Qt5Agg


In [2]:
from scipy.ndimage.filters import median_filter

In [12]:
from Experimentos import Experimentos, labelLines

In [25]:
#for i, exp in enumerate(Experimentos):
#    print(i, exp, (exp.flir.dtypes if exp.flir is not None else 0))

In [4]:
Experimentos[14].flir.describe()

,FLIR: tiempo,FLIR: area,FLIR: rango 1,FLIR: rango 2,FLIR: rango 3,FLIR: rango 4,FLIR: rango 5,FLIR: rango 6,FLIR: rango 7,FLIR: rango 8,FLIR: rango 9,moving average,moving std
count,1557.000000,1557.000000,1557.000000,1557.000000,1557.000000,1557.000000,1557.000000,1557.000000,1557.000000,1557.000000,1557.000000,1557.000000,1557.000000
mean,785.181118,94162.901092,0.112485,0.120258,0.211461,0.182675,0.184660,0.087382,0.063736,0.033939,0.003378,94155.919717,469.647036
std,455.769516,20014.599567,0.085897,0.103205,0.164399,0.154823,0.228621,0.157622,0.146061,0.101394,0.016267,19989.444232,907.503220
min,0.000000,299.000000,0.015796,0.005306,0.005876,0.000030,0.000000,0.000000,0.000000,0.000000,0.000000,6333.500000,38.071816
25%,391.000000,95783.000000,0.032509,0.020066,0.038780,0.029736,0.000000,0.000000,0.000000,0.000000,0.000000,95764.000000,92.242586
50%,781.000000,101901.000000,0.085578,0.100824,0.189249,0.152571,0.043643,0.000000,0.000000,0.000000,0.000000,101891.500000,123.731405
75%,1180.000000,103825.000000,0.188446,0.195695,0.389261,0.331465,0.344386,0.103819,0.000000,0.000000,0.000000,103799.000000,220.558973
max,1578.000000,104805.000000,0.327759,0.354756,0.443221,0.450265,0.653564,0.575844,0.591025,0.527886,0.124881,104703.500000,5446.076358


In [5]:
numeropixeles = 1020100
rho_manjar = 1340
kilos_manjar = 2

In [6]:
for exp in Experimentos:
    if exp.flir is not None:
        try:
            exp.flir['area_m2'] = exp.flir['FLIR: area']/numeropixeles
            exp.flir['volumen_emitido'] = exp.param['caudal'][0]*exp.flir['FLIR: tiempo']
        except:
            continue
        volumen_total = kilos_manjar/rho_manjar
        exp.flir['volumen_emitido'][exp.flir['volumen_emitido'] > volumen_total] = volumen_total
        exp.flir['espesor'] = 3*exp.flir['volumen_emitido']/(2*exp.flir['area_m2'])

C:\Users\CCVV\Anaconda3b\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [29]:
exp.get_ranges()

[24.0,
 27.377777777777776,
 30.755555555555556,
 34.133333333333333,
 37.511111111111113,
 40.888888888888886,
 44.266666666666666,
 47.644444444444446,
 51.022222222222226,
 54.399999999999999]

In [46]:
#FALTA: agregar rango de temperaturas a partir de archivo "resumen general experimentos":
#(max-min)/rangos = salto temp
#rangos=min+salto etc

In [ ]:
#FALTA: graficar tiempo en que prevalece cada rango de temperatura? y como relacionarlo a la velocidad

In [30]:
exp = Experimentos[30]

In [31]:
Experimentos[6].get_closest_points_to_intersections()

['0.9536492014866595', '0.059804062054914814', '1.0332272690781028', '-0.7738854591314837', '0.6353865292881408', '0.6837549563979712', '0.5408321558488145']
[[ inf  inf  inf  inf  inf  inf  inf]
 [ inf  inf  inf  inf  inf  inf  inf]
 [ inf  inf  inf  inf  inf  inf  inf]
 [ inf  inf  inf  inf  inf  inf  inf]
 [ inf  inf  inf  inf  inf  inf  inf]
 [ inf  inf  inf  inf  inf  inf  inf]
 [ inf  inf  inf  inf  inf  inf  inf]]


C:\Users\CCVV\Anaconda3b\lib\site-packages\scipy\optimize\minpack.py:715: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


(0.63538652928814077,
 0.68375495639797124,
 0.5408321558488145,
 (-4.8117844353510897, -4.4228486291941369),
 (-2.5523293261054341, -2.9957322735539909))

In [32]:
for exp in Experimentos:
    if exp.flir is not None:
        try:
            exp.flir['rango 1'] = exp.flir['FLIR: rango 1']+exp.flir['FLIR: rango 2']+exp.flir['FLIR: rango 3']
            exp.flir['rango 2'] = exp.flir['FLIR: rango 4']+exp.flir['FLIR: rango 5']+exp.flir['FLIR: rango 6']
            exp.flir['rango 3'] = exp.flir['FLIR: rango 7']+exp.flir['FLIR: rango 8']+exp.flir['FLIR: rango 9']

        except:
            continue
#Experimentos[30].flir['rango 1'] = Experimentos[30].flir['FLIR: rango 1']+Experimentos[30].flir['FLIR: rango 2']+Experimentos[30].flir['FLIR: rango 3']
#Experimentos[30].flir['rango 2'] = Experimentos[30].flir['FLIR: rango 4']+Experimentos[30].flir['FLIR: rango 5']+Experimentos[30].flir['FLIR: rango 6']
#Experimentos[30].flir['rango 3'] = Experimentos[30].flir['FLIR: rango 7']+Experimentos[30].flir['FLIR: rango 8']+Experimentos[30].flir['FLIR: rango 9']


In [27]:
Experimentos[30].flir.describe()

,FLIR: tiempo,FLIR: area,FLIR: rango 1,FLIR: rango 2,FLIR: rango 3,FLIR: rango 4,FLIR: rango 5,FLIR: rango 6,FLIR: rango 7,FLIR: rango 8,FLIR: rango 9,moving average,moving std,area_m2,volumen_emitido,espesor,rango 1,rango 2,rango 3
count,1948.000000,1948.000000,1948.000000,1948.000000,1948.000000,1948.000000,1948.000000,1948.000000,1948.000000,1948.000000,1948.000000,1948.000000,1948.000000,1948.000000,1948.000000,1948.000000,1948.000000,1948.000000,1948.000000
mean,992.003080,87628.585216,0.157066,0.244931,0.322767,0.169875,0.061603,0.027425,0.007986,0.006875,0.001472,87629.097279,623.995370,0.085902,0.001435,0.025218,0.724764,0.258903,0.016333
std,578.671287,16461.360387,0.093808,0.178898,0.188563,0.230611,0.152577,0.100385,0.054138,0.059602,0.019609,16407.960061,2359.663421,0.016137,0.000233,0.002532,0.354230,0.342575,0.101643
min,0.000000,1666.000000,0.032709,0.009080,0.007050,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8818.500000,75.340263,0.001633,0.000000,0.000000,0.054479,0.000000,0.000000
25%,488.750000,86830.500000,0.062744,0.069849,0.169897,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,86810.500000,126.767576,0.085120,0.001493,0.023728,0.436610,0.000000,0.000000
50%,987.500000,94123.000000,0.149057,0.232019,0.340263,0.005753,0.000000,0.000000,0.000000,0.000000,0.000000,94106.250000,166.148993,0.092268,0.001493,0.024203,0.994247,0.005753,0.000000
75%,1493.250000,96188.750000,0.242202,0.401451,0.476966,0.366491,0.000060,0.000000,0.000000,0.000000,0.000000,96165.000000,255.760055,0.094293,0.001493,0.026042,1.000000,0.506228,0.000000
max,1998.000000,97414.000000,0.587635,0.584223,0.624769,0.647803,0.817099,0.787302,0.610024,0.736516,0.315819,97111.000000,23362.983847,0.095495,0.001493,0.033482,1.000000,0.943460,0.835539


In [7]:
#GENERA PATCHES DE COLORES SEGUN TEMPERATURA
import matplotlib.patches as mpatches

pat1 = mpatches.Patch(color='C1', label='temp < 40')
pat2 = mpatches.Patch(color='C2', label='temp < 45')
pat3 = mpatches.Patch(color='C3', label='temp < 50')
pat4 = mpatches.Patch(color='C4', label='temp < 55')
pat5 = mpatches.Patch(color='C5', label='temp < 60')
pat6 = mpatches.Patch(color='C6', label='temp < 65')
pat7 = mpatches.Patch(color='C7', label='temp < 70')

numerocolores = 7
colors = ['pat{}'.format(i+1) for i in range(numerocolores)]

print(colors)

['pat1', 'pat2', 'pat3', 'pat4', 'pat5', 'pat6', 'pat7']


In [8]:
#LISTA para ESPESOR vs TIEMPO
listaporcaudal = []

for exp in Experimentos:
    if exp.flir is not None:
        try:
            holder = {} #es un diccionario 
            holder['nombre'] = exp.param['NOMBRE'][0]#solo el valor de la primera fila
            holder['espesor'] = exp.flir['espesor']#toda la columna
            holder['tiempo'] = exp.flir['FLIR: tiempo']#toda la columna
            holder['caudal'] = exp.param['caudal'][0]#solo el valor de la primera fila
            holder['angulo'] = exp.param['angulo inclinacion'][0]
            holder['uso'] = exp.param['uso manjar'][0]

            listaporcaudal.append(holder)
        except Exception as e:
            print(e)

In [38]:
#for exp in Experimentos:
#    print(exp.flir['espesor'] if exp.flir is not None else None)

In [9]:
#crear GRUPOS segun CAUDAL
from itertools import groupby
count = 0

#key es la lista donde estan los diccionarios de exp y DF
#group es que se agrupa segun el valor de 'caudal' en este caso

for key, group in groupby(sorted(listaporcaudal, key=lambda x: x['caudal']), key=lambda x: x['caudal']):
    print(key)
    count += 1
    #for exp in group:
        #muestra el nombre del grupo y los elementos que tiene
        #print(exp['nombre']) 

1e-06
5e-06
1e-05
1.7e-05
2.5e-05


In [19]:
#ESPESOR vs TIEMPO agrupados por caudales
fig = plt.figure()
axes = [fig.add_subplot(3,2,i+1) for i in range(count)]
counter = 0

for key, group in groupby(sorted(listaporcaudal, key=lambda x: x['caudal']), key=lambda x: x['caudal']):
    print(key)
    c = 0
    ax = axes[counter]
    #ax.set_xlim(minimoejelargo, maximoejelargo)
    #ax.set_ylim(0, maximoejeespesor)
    ax.set_title('{}cc'.format(key*1000000))
    counter += 1
    linea1= ax.plot([],[], label='15°', linestyle='--', color='black')
    linea2= ax.plot([],[], label='10°', color='black')
    
    for exp in group:
        #print("C{}".format(exp["rango temperatura"]))
        #if exp['uso'] == 1:
            c = c + 1
            if exp['angulo'] == 15:
                ax.plot(exp['tiempo'], exp['espesor'], linestyle= '--')
            else:
                ax.plot(exp['tiempo'], exp['espesor'], label = '')
        
legend1 = plt.legend(handles=[pat1, pat2, pat3, pat4, pat5, pat6, pat7], bbox_to_anchor=(1.8, 0.5), loc=5)
plt.legend(bbox_to_anchor=(1.8, 0.5), loc=6)
plt.gca().add_artist(legend1)
fig.show()

1e-06
5e-06
1e-05
1.7e-05
2.5e-05


In [20]:
#LISTA para AREA vs TIEMPO
otralistaporcaudal = []

for exp in Experimentos:
    if exp.flir is not None:
        try:
            holder = {} #es un diccionario 
            holder['nombre'] = exp.param['NOMBRE'][0]#solo el valor de la primera fila
            holder['area'] = exp.flir['area_m2']#toda la columna
            holder['tiempo'] = exp.flir['FLIR: tiempo']#toda la columna
            holder['caudal'] = exp.param['caudal'][0]#solo el valor de la primera fila
            holder['angulo'] = exp.param['angulo inclinacion'][0]
            holder['uso'] = exp.param['uso manjar'][0]

            otralistaporcaudal.append(holder)
        except:
            continue

In [23]:
#AREA vs TIEMPO agrupados por caudales
fig = plt.figure()
axes = [fig.add_subplot(3,2,i+1) for i in range(count)]
counter = 0

for key, group in groupby(sorted(otralistaporcaudal, key=lambda x: x['caudal']), key=lambda x: x['caudal']):
    print(key)
    ax = axes[counter]
    #ax.set_xlim(minimoejelargo, maximoejelargo)
    #ax.set_ylim(0, maximoejeespesor)
    ax.set_title('{}cc'.format(key*1000000))
    counter += 1
    
    for exp in group:
        ax.plot(exp['tiempo'], exp['area'])
        
legend1 = plt.legend(handles=[pat1, pat2, pat3, pat4, pat5, pat6, pat7], bbox_to_anchor=(1.8, 0.5), loc=5)
#plt.legend(bbox_to_anchor=(1.8, 0.5), loc=6)
plt.gca().add_artist(legend1)
fig.show()

1e-06
5e-06
1e-05
1.7e-05
2.5e-05


In [13]:
for k, g in groupby(sorted(otralistaporcaudal, key=lambda x: x['caudal']), key=lambda x: x['caudal']):
    print(k, g)